## WHO GHO indicators

In [19]:
# import libraries (installed in epi-core environment)
import pandas as pd
import requests
import sqlalchemy
from sqlalchemy import create_engine, text, URL
import json
from pathlib import Path
from dotenv import load_dotenv
import os

In [20]:
# check for .env

env_path = Path.cwd().parent / "env" / ".env"  # up one folder

if not os.path.exists(env_path):
    sys.exit(f"Missing {env_path} file. Please create one with your DB credentials.")

load_dotenv(dotenv_path=env_path) 

PG_USER = os.getenv("PGUSER")
PG_PASSWORD = os.getenv("PGPASSWORD")
PG_HOST = os.getenv("PGHOST")
PG_PORT = os.getenv("PGPORT")
PG_DATABASE = os.getenv("PGDATABASE")

# Validate
missing_vars = [var for var, val in {
    "PG_USER": PG_USER,
    "PG_PASSWORD": PG_PASSWORD,
    "PG_HOST": PG_HOST,
    "PG_PORT": PG_PORT,
    "PG_DATABASE": PG_DATABASE
}.items() if val is None]

if missing_vars:
    print(f"Missing required environment variables: {', '.join(missing_vars)} "
             f"\nMake sure they are defined in {env_path}")

In [21]:
# configuration

pg_url = URL.create(
    "postgresql+psycopg",
    username=os.getenv("PGUSER"),
    password=os.getenv("PGPASSWORD"),
    host=os.getenv("PGHOST", "localhost"),
    port=int(os.getenv("PGPORT", 5432)),
    database=os.getenv("PGDATABASE"),
)

INDICATORS = ["WHOSIS_000001", "WHOSIS_000002", "MDG_0000000026", "u5mr", "MDG_0000000020"]
BASE_URL = "https://ghoapi.azureedge.net/api"  # WHO GHO OData endpoint

In [22]:
# connect to db
engine = create_engine(pg_url, pool_pre_ping=True)

with engine.connect() as conn:
    # Drop existing tables if you want a clean run
    conn.execute(text("""
    DO $$ DECLARE r RECORD;
    BEGIN
        FOR r IN (SELECT tablename FROM pg_tables WHERE schemaname = 'public') LOOP
            EXECUTE 'DROP TABLE IF EXISTS public.' || quote_ident(r.tablename) || ' CASCADE';
        END LOOP;
    END $$;
    """))
    conn.commit()

In [23]:
# create schema by running sql file

def run_sql_file(engine, path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"SQL file not found: {path}")
    with open(path, "r", encoding="utf-8") as f:
        sql_text = f.read()
    # exec_driver_sql allows multiple statements separated by semicolons
    with engine.begin() as conn:      # begins a transaction
        conn.exec_driver_sql(sql_text)


try:
    run_sql_file(engine, os.path.join("..", "scripts", "schema.sql") if os.path.basename(os.getcwd())=="notebooks" else "scripts/schema.sql")
    print("Schema created/verified.")
except Exception as e:
    print("Failed to apply SQL files:", e)
    raise

Schema created/verified.


In [30]:
# extract

all_records = []

for ind in INDICATORS:
    print(f"Fetching {ind}...")
    url = f"{BASE_URL}/{ind}"
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"Failed to fetch {ind}: {resp.status_code}")
        continue
    data = resp.json().get("value", [])
    for row in data:
        all_records.append({
            "indicator_id": ind,
            "indicator_name": row.get("Indicator"),
            "country_code": row.get("SpatialDim"),
            "country_name": row.get("Location"),  # WHO uses SpatialDimType as country name for some datasets
            "region": row.get("ParentLocation"),
            "year": int(row.get("TimeDim")) if row.get("TimeDim") else None,
            "value": float(row.get("NumericValue")) if row.get("NumericValue") else None
        })

df = pd.DataFrame(all_records)

Fetching WHOSIS_000001...
Fetching WHOSIS_000002...
Fetching MDG_0000000026...
Fetching u5mr...
Fetching MDG_0000000020...


In [31]:
# transform

# Clean up country_name (SpatialDimType is often 'COUNTRY', so we'll fetch proper country names from WHO "Dimension" endpoint)
# For now, we'll just map code -> itself as placeholder
df['country_name'] = df['country_code']

# Split into dimension + fact tables
dim_indicator = df[['indicator_id', 'indicator_name']].drop_duplicates()
dim_country = df[['country_code', 'country_name', 'region']].drop_duplicates()
fact_measure = df[['indicator_id', 'country_code', 'year', 'value']]


In [32]:
df.head()

,indicator_id,indicator_name,country_code,country_name,region,year,value
0,WHOSIS_000001,None,BGD,BGD,South-East Asia,2006,70.280734
1,WHOSIS_000001,None,NLD,NLD,Europe,2004,81.217075
2,WHOSIS_000001,None,CZE,CZE,Europe,2010,74.406599
3,WHOSIS_000001,None,ALB,ALB,Europe,2007,74.536786
4,WHOSIS_000001,None,NER,NER,Africa,2021,59.970705


In [9]:
# load
dim_indicator.to_sql('dim_indicator', engine, if_exists='append', index=False)

-1

In [10]:
dim_country.to_sql('dim_country', engine, if_exists='append', index=False)

-1

In [18]:
print(fact_measure[(fact_measure["country_code"] == "IRL") & (fact_measure["year"] == 2005)])

         indicator_id country_code  year      value
39      WHOSIS_000001          IRL  2005  81.096466
137     WHOSIS_000001          IRL  2005  76.510633
9268    WHOSIS_000001          IRL  2005  78.820083
16507   WHOSIS_000002          IRL  2005  68.798188
18514   WHOSIS_000002          IRL  2005  67.960814
25466   WHOSIS_000002          IRL  2005  67.087436
28888  MDG_0000000026          IRL  2005   6.334765
52997  MDG_0000000020          IRL  2005  12.000000


In [13]:
with engine.begin() as conn:
    try:
        fact_measure.to_sql(
            'fact_measure', conn, if_exists='append', index=False, method='multi', chunksize=1000
        )
    except Exception as e:
        print("INSERT FAILED:", e)  # look at the *first* error printed above
        raise

INSERT FAILED: (psycopg.errors.UniqueViolation) duplicate key value violates unique constraint "uq_fact"
DETAIL:  Key (indicator_id, country_code, year)=(WHOSIS_000001, IRL, 2005) already exists.
[SQL: INSERT INTO fact_measure (indicator_id, country_code, year, value) VALUES (%(indicator_id_m0)s::VARCHAR, %(country_code_m0)s::VARCHAR, %(year_m0)s::BIGINT, %(value_m0)s), (%(indicator_id_m1)s::VARCHAR, %(country_code_m1)s::VARCHAR, %(year_m1)s::BIGINT, %(value_m1)s), (%(indicator_id_m2)s::VARCHAR, %(country_code_m2)s::VARCHAR, %(year_m2)s::BIGINT, %(value_m2)s), (%(indicator_id_m3)s::VARCHAR, %(country_code_m3)s::VARCHAR, %(year_m3)s::BIGINT, %(value_m3)s), (%(indicator_id_m4)s::VARCHAR, %(country_code_m4)s::VARCHAR, %(year_m4)s::BIGINT, %(value_m4)s), (%(indicator_id_m5)s::VARCHAR, %(country_code_m5)s::VARCHAR, %(year_m5)s::BIGINT, %(value_m5)s), (%(indicator_id_m6)s::VARCHAR, %(country_code_m6)s::VARCHAR, %(year_m6)s::BIGINT, %(value_m6)s), (%(indicator_id_m7)s::VARCHAR, %(country_code

IntegrityError: (psycopg.errors.UniqueViolation) duplicate key value violates unique constraint "uq_fact"
DETAIL:  Key (indicator_id, country_code, year)=(WHOSIS_000001, IRL, 2005) already exists.
[SQL: INSERT INTO fact_measure (indicator_id, country_code, year, value) VALUES (%(indicator_id_m0)s::VARCHAR, %(country_code_m0)s::VARCHAR, %(year_m0)s::BIGINT, %(value_m0)s), (%(indicator_id_m1)s::VARCHAR, %(country_code_m1)s::VARCHAR, %(year_m1)s::BIGINT, %(value_m1)s), (%(indicator_id_m2)s::VARCHAR, %(country_code_m2)s::VARCHAR, %(year_m2)s::BIGINT, %(value_m2)s), (%(indicator_id_m3)s::VARCHAR, %(country_code_m3)s::VARCHAR, %(year_m3)s::BIGINT, %(value_m3)s), (%(indicator_id_m4)s::VARCHAR, %(country_code_m4)s::VARCHAR, %(year_m4)s::BIGINT, %(value_m4)s), (%(indicator_id_m5)s::VARCHAR, %(country_code_m5)s::VARCHAR, %(year_m5)s::BIGINT, %(value_m5)s), (%(indicator_id_m6)s::VARCHAR, %(country_code_m6)s::VARCHAR, %(year_m6)s::BIGINT, %(value_m6)s), (%(indicator_id_m7)s::VARCHAR, %(country_cod

In [11]:
fact_measure.to_sql('fact_measure', engine, if_exists='append', index=False)

print("ETL complete. Data loaded into epidemiology database.")
print(f"Indicators loaded: {len(dim_indicator)} | Countries: {len(dim_country)} | Records: {len(fact_measure)}")

error ignored terminating <psycopg.Pipeline [INERROR, pipeline=ON] (host=localhost user=postgres database=epidemiology) at 0x1eeec97e0d0>: pipeline aborted


IntegrityError: (psycopg.errors.UniqueViolation) duplicate key value violates unique constraint "uq_fact"
DETAIL:  Key (indicator_id, country_code, year)=(WHOSIS_000001, IRL, 2005) already exists.
[SQL: INSERT INTO fact_measure (indicator_id, country_code, year, value) VALUES (%(indicator_id)s::VARCHAR, %(country_code)s::VARCHAR, %(year)s::BIGINT, %(value)s)]
[parameters: [{'indicator_id': 'WHOSIS_000001', 'country_code': 'BGD', 'year': 2006, 'value': 70.28073443}, {'indicator_id': 'WHOSIS_000001', 'country_code': 'NLD', 'year': 2004, 'value': 81.21707544}, {'indicator_id': 'WHOSIS_000001', 'country_code': 'CZE', 'year': 2010, 'value': 74.4065987}, {'indicator_id': 'WHOSIS_000001', 'country_code': 'ALB', 'year': 2007, 'value': 74.53678554}, {'indicator_id': 'WHOSIS_000001', 'country_code': 'NER', 'year': 2021, 'value': 59.97070525}, {'indicator_id': 'WHOSIS_000001', 'country_code': 'TZA', 'year': 2004, 'value': 56.56155012}, {'indicator_id': 'WHOSIS_000001', 'country_code': 'ZMB', 'yea